# extract_images.ipynb

Run this notebook using the **Python 3** kernel.

This notebook uses the ImageJ particle analyzer to extract individual insect images from a photo of a sticky trap.

## Papermill

Can be run using **paperemill**. Example:
```
papermill -p IMAGE_PATH 2021-11-24/PB240063.JPG extract_images.ipynb out.ipynb
```

## Output directory structure

If the sticky trap image is in **a_file_path/my_sticky_trap.jpg**, the following file structure will be generated.

```
my_sticky_trap (dir)
   my_sticky_trap.jpg (copied from original file)
   my_stick_trap.csv (metadata generated particle analyzer)
   extracted_images (dir)
      my_sticky_trap_001.jpg
      my_sticky_trap_002.jpg
      ...
```

## ImageJ

**pa1.ijm**
```
run("8-bit");
run("Bandpass Filter...", "filter_large=40 filter_small=3 suppress=None tolerance=5 autoscale saturate");
setAutoThreshold("Triangle");
run("Analyze Particles...", "size=1000-Infinity circularity=[] display exclude");
saveAs("Results", "/home/aubrey/Desktop/bug_soup/Results.csv");
eval("script", "System.exit(0);");
```

**Command line:**
```
/home/aubrey/Fiji.app/ImageJ-linux64 P5150004.JPG -macro pa.ijm
```


# References

https://www.youtube.com/watch?v=ayQSqdOPR-c

In [1]:
import pandas as pd
import cv2
import os
import shutil
import numpy as np

In [2]:
# Parameters: may be used by papermill

RESULTS_PATH = '/home/aubrey/Desktop/bug_soup/Results.csv'
OUTPUT_DIR = 'output'
IMAGEJ_PATH = '/home/aubrey/Fiji.app/ImageJ-linux64'
IMAGE_PATH = '/home/aubrey/Desktop/bug_soup/2021-11-24/PB240063.JPG'
MARGIN = 40

In [3]:
# Create OUTPUT_DIR if it does not already exist

if not os.path.exists(OUTPUT_DIR):
     os.mkdir(OUTPUT_DIR)

# Create or recreate a folder for the image in OUTPUT_DIR

image_file_name = os.path.basename(IMAGE_PATH)
image_output_dir = f'{OUTPUT_DIR}/{image_file_name.split(".")[0]}'
print(image_output_dir)
shutil.rmtree(image_output_dir, ignore_errors=True)
os.mkdir(image_output_dir)

# Create folder for extracted images

os.mkdir(f'{image_output_dir}/extracted_images')

# Copy image into the new folder

os.system(f'cp {IMAGE_PATH} {image_output_dir}/{image_file_name}')  

# Perform particle analysis using ImageJ

image_path = f'{image_output_dir}/{image_file_name}'
command_line = f'{IMAGEJ_PATH} {image_path} -macro pa1.ijm'
print(command_line)
os.system(command_line)

output/PB240063
/home/aubrey/Fiji.app/ImageJ-linux64 output/PB240063/PB240063.JPG -macro pa1.ijm


0

In [4]:
df = pd.read_csv(RESULTS_PATH)
df.rename(columns={' ': 'i'}, inplace=True)
df.sort_values('Area', inplace=True)
df['i'] = np.arange(1,df.shape[0]+1)
csv_path = f'{image_path.split(".")[0]}.csv'
df.to_csv(csv_path, index=False)
df

,i,Area,Mean,StdDev,Min,Max,X,Y,Perim.,BX,...,Angle,Circ.,Feret,FeretX,FeretY,FeretAngle,MinFeret,AR,Round,Solidity
66,1,1001,24.380,38.553,0,133,841.165,865.212,120.953,823,...,157.052,0.860,40.853,824,856,158.459,35.355,1.117,0.895,0.949
173,2,1001,112.421,15.262,61,133,355.070,2885.213,210.936,330,...,57.794,0.283,59.506,342,2909,65.158,38.626,2.132,0.469,0.625
150,3,1006,107.383,21.115,26,133,3089.907,2516.156,280.492,3070,...,101.383,0.161,78.313,3091,2482,95.128,34.785,3.293,0.304,0.522
101,4,1010,130.754,2.821,115,133,3298.433,1536.836,342.475,3270,...,119.882,0.108,127.016,3279,1480,117.170,28.178,7.248,0.138,0.363
30,5,1012,99.994,24.399,16,133,2000.404,206.789,244.978,1983,...,95.929,0.212,69.778,1993,184,115.463,40.954,1.845,0.542,0.508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,198,57557,90.040,35.952,0,133,1621.003,900.154,7600.488,1320,...,83.315,0.013,745.870,1326,1233,75.163,478.254,1.532,0.653,0.227
155,199,71289,109.331,25.287,0,133,2364.173,2757.073,9079.792,1939,...,11.067,0.011,842.784,1939,2910,18.972,387.688,2.765,0.362,0.268
84,200,86382,93.354,29.567,0,133,2127.076,1541.138,12870.139,1669,...,130.326,0.007,1010.030,1826,1223,130.503,774.870,1.451,0.689,0.169
1,201,126182,100.727,28.176,0,133,1377.179,425.090,12449.085,979,...,168.300,0.010,928.130,1071,42,145.402,702.695,1.120,0.893,0.270


In [5]:
# Use bounding box values to extract images.
# Images with bounding boxes plus margin which exceed the original image edges are excluded.

image = cv2.imread(image_path)
ymax = image.shape[0] - 1
xmax = image.shape[1] - 1

for row in df.itertuples():
    y1 = row.BY - MARGIN
    y2 = row.BY + row.Height + MARGIN
    x1 = row.BX - MARGIN
    x2 = row.BX + row.Width + MARGIN
    if (y1 >= 0) and (y2 <= ymax) and (x1 >= 0) and (x2 <= xmax):
        roi = image[y1:y2, x1:x2]
        filename = f'{image_output_dir}/extracted_images/{image_file_name.split(".")[0]}_{row.i:03}.jpg'
        print(filename)
        cv2.imwrite(filename, roi)

output/PB240063/extracted_images/PB240063_001.jpg
output/PB240063/extracted_images/PB240063_002.jpg
output/PB240063/extracted_images/PB240063_003.jpg
output/PB240063/extracted_images/PB240063_004.jpg
output/PB240063/extracted_images/PB240063_005.jpg
output/PB240063/extracted_images/PB240063_006.jpg
output/PB240063/extracted_images/PB240063_007.jpg
output/PB240063/extracted_images/PB240063_008.jpg
output/PB240063/extracted_images/PB240063_009.jpg
output/PB240063/extracted_images/PB240063_010.jpg
output/PB240063/extracted_images/PB240063_012.jpg
output/PB240063/extracted_images/PB240063_013.jpg
output/PB240063/extracted_images/PB240063_014.jpg
output/PB240063/extracted_images/PB240063_015.jpg
output/PB240063/extracted_images/PB240063_017.jpg
output/PB240063/extracted_images/PB240063_018.jpg
output/PB240063/extracted_images/PB240063_019.jpg
output/PB240063/extracted_images/PB240063_020.jpg
output/PB240063/extracted_images/PB240063_021.jpg
output/PB240063/extracted_images/PB240063_023.jpg
